In [5]:
import requests
from bs4 import BeautifulSoup
import string
import re
import time
import datetime
from datetime import date, timedelta
import time

import numpy as np
import pandas as pd
import unicodecsv as csv

import os
import json
import glob
import zipfile
import wget

# <center> Download dos dados de Bilhetagem da SPTrans </center>

In [6]:
diretorio = "C:\\Users\\f112596\\Desktop\\Projetos\\DO\\Lab Transp" #"C:/Users/usuario/Pasta Raiz"

In [7]:
"""
DEFINIÇÃO DAS ROTINAS A SEREM EXECUTADAS:
"""
Download_BD_Atualizacao = 0
Download_BD_Historico = 1

In [8]:
class Classe_Geral(object):
    #Crawler
    url_principal = "https://capital.sp.gov.br/web/mobilidade/w/passageiros-transportados-2025"
    url_2014 = "https://www.prefeitura.sp.gov.br/cidade/secretarias/mobilidade/institucional/sptrans/acesso_a_informacao/index.php?p=152417"
    url_2013 = "https://www.prefeitura.sp.gov.br/cidade/secretarias/mobilidade/institucional/sptrans/acesso_a_informacao/index.php?p=164379"

    def carregar_Anos_Consulta(self):
        """
        Este método carrega os anos para os quais
        existem dados disponíves na url_principal
        """
        r1 = requests.get(self.url_principal)
        coverpage = r1.content
        soup = BeautifulSoup(coverpage, 'html5lib')


        result_set = soup.find_all("div", {"class": "psp-agencies-content__content"})
        anos_disponiveis = result_set[0].find_all("a")

        ano = 0
        lista_anos = []        
        lista_anos.append(2025) #url Principal
        
        for elem in anos_disponiveis:
            try:
                ano = int(elem.get_text())
                if ano >= 2000:
                    lista_anos.append(ano)
            except:
                pass

        return lista_anos, result_set
    
    def criar_Relacao_Datas(self, lista_anos):
        """
        Para garantir que todos os arquivos sejam baixados
        e validar a consistência do banco de dados,
        crio uma relação de datas, que deverão ser checadas:
                                                            i) no banco e 
                                                            ii) na página para download    
        """
        data_base = datetime.date.today()
        data_inicial = datetime.date(np.min(lista_anos), 1, 1)

        lista_datas = [data_base - timedelta(days=x) for x in range((data_base-data_inicial).days)]
        
        
        #Montagem do dicionário de log
        dict_log_datas = {}
        for data in lista_datas:
            dict_log_datas[data] = "Não Baixado"

        return dict_log_datas
    
    def criar_Relacao_Datas_Atualizacao(self, dict_log_datas):
        """
        Para garantir que todos os arquivos sejam baixados
        e validar a consistência do banco de dados,
        crio uma relação de datas, que deverão ser checadas:
                                                            i) no banco e 
                                                            ii) na página para download    
        """
        data_base = datetime.date.today()
        
        chave_ultimo_download = [k for k, v in dict_log_datas.items() if v[1] == 'Baixado'][0]
        data_ultimo_download = datetime.datetime.strptime(dict_log_datas[chave_ultimo_download][0], '%Y-%m-%d')
        data_inicial = (data_ultimo_download + datetime.timedelta(days=1)).date()
        
        lista_datas = [data_base - timedelta(days=x) for x in range((data_base-data_inicial).days)]
        
        dict_log_datas = {}
        for data in lista_datas:
            dict_log_datas[data] = "Não Baixado"

        return dict_log_datas
    
    
    def carregar_Href_Ano(self, ano, result_set):
        """
        Carregar o href do ano selecionado
        """ 
        if ano == '2014':
            return self.url_2014
        
        elif ano == "2013":
            return self.url_2013

        elif ano == "2025":
            return self.url_principal        
        
        else:
            for elem in result_set[0].find_all("a"):
                if elem.get_text().strip() == ano:
                    if elem['href'][0:6] != "https:": 
                        url = "https://capital.sp.gov.br" + elem['href']
                    else:
                        url = elem['href']
                    
                    return url 
            
           
    def carregar_Href_datas(self, url):
        """
        Com base em uma url do ano seleciona
        realizada as buscas das urls dos arquivos
        """
        r1 = requests.get(url)
        coverpage = r1.content
        soup = BeautifulSoup(coverpage, 'html5lib')

        result_set = soup.find_all("td")

        lista_url_arquivos = []
        for elem in result_set:
            if elem.get_text() not in ['Dom', 'Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Total', '\xa0']:
                try:
                    if "Consolidado" not in elem.find_all("a")[0]['href']:
                        lista_url_arquivos.append(elem.find_all("a")[0]['href'])
                except:
                    pass

        return lista_url_arquivos

In [9]:
class gerenciador_Download(object):    
    
    def criar_Chave_Meses(self):
        """
        As urls da SPTRANS vêm com o padrão de mês MMM e eu preciso de um int
        Por isso uso esse dict
        """
        dict_chave_meses = {"JAN" : 1,
                            "FEV" : 2,
                            "MAR" : 3,
                            "ABR" : 4,
                            "MAI" : 5,
                            "JUN" : 6,
                            "JUL" : 7,
                            "AGO" : 8,
                            "SET" : 9,
                            "OUT" : 10,
                            "NOV" : 11,
                            "DEZ" : 12}  
        return dict_chave_meses
    
    
    
    def mapeamento_Arquivos(self, lista_arquivos, ano):
        """
        Defino a relação de arquivos disponíveis, criando um dicionário, 
        cuja chave é o nome/data do arquivo        
        """

        dict_chave_meses = self.criar_Chave_Meses()

        dict_arquivos_disponiveis = {}    
        
        for arquivo in lista_arquivos:
            inicio = arquivo.find("upload/") + len("upload/")
            final = arquivo.find(".xls")

            #Formato: DDMMMAAAA ou "ddmmaa(1)"
            if (len(arquivo[inicio : final]) == 9)  or (len(arquivo[inicio : final]) == 12):
                try:
                    data_ref_arquivo = datetime.date(int(arquivo[inicio : final][5:9]),
                                                     dict_chave_meses[arquivo[inicio : final][2:5]],
                                                     int(arquivo[inicio : final][0:2]))    
                except:
                    novo_final = arquivo[inicio : final].find("(")
                    data_ref_arquivo = datetime.date(int(str(20) + arquivo[inicio : inicio + novo_final][4:6]),
                                                     int(arquivo[inicio : inicio + novo_final][2:4]),
                                                     int(arquivo[inicio : inicio + novo_final][0:2]))                    
            
            #Formato: DDMMAAAA
            elif (len(arquivo[inicio : final]) == 11):
                data_ref_arquivo = datetime.date(int(arquivo[inicio : final][4:8]),
                                     int(arquivo[inicio : final][2:4]),
                                     int(arquivo[inicio : final][0:2]))
            
            #Formato: ddmmaaaa
            elif len(arquivo[inicio : final]) == 8:
                data_ref_arquivo = datetime.date(int(arquivo[inicio : final][4:9]),
                                             int(arquivo[inicio : final][2:4]),                                 
                                             int(arquivo[inicio : final][0:2]))


            #Formato: "ddmmaaaa_Dados para Transparência"
            elif (len(arquivo[inicio : final]) == 33) or (len(arquivo[inicio : final]) == 36):
                novo_final = arquivo[inicio : final].find("_Dados para Transparência")
                data_ref_arquivo = datetime.date(int(arquivo[inicio : inicio + novo_final][4:9]),
                                         int(arquivo[inicio : inicio + novo_final][2:4]),                                 
                                         int(arquivo[inicio : inicio + novo_final][0:2]))

            #Formato: DD-MMM ou "ddmmaa"
            elif len(arquivo[inicio : final]) == 6:
                try:
                    data_ref_arquivo = datetime.date(ano,
                                                 dict_chave_meses[arquivo[inicio : final][3:6]],
                                                 int(arquivo[inicio : final][0:2]))
                
                except:                
                    data_ref_arquivo = datetime.date(int(str(20) + arquivo[inicio : final][4:6]),
                                                 int(arquivo[inicio : final][2:4]),
                                                 int(arquivo[inicio : final][0:2]))             
                
                
            #Formato: "Passag-AAAAMMDD"
            elif len(arquivo[inicio : final]) == 15 or len(arquivo[inicio : final]) == 18:
                novo_inicio = inicio + len("Passag-")    
                data_ref_arquivo = datetime.date(int(arquivo[novo_inicio : final][0:4]),
                                                 int(arquivo[novo_inicio : final][4:6]),
                                                 int(arquivo[novo_inicio : final][6:8]))      

                
                
            #Formato: "ddmmaaaa_Dados para Transparência"
                # no subditerório 'upload/transportes/SPTrans/acesso_a_informacao/2017/janeiro/passageiros/'
            elif (len(arquivo[inicio : final]) >= 77 and len(arquivo[inicio : final]) <= 82) or ( len(arquivo[inicio : final]) in [57, 58, 60, 62]):
                try:
                    novo_inicio = arquivo.find("passageiros/") + len("passageiros/")
                    novo_inicio_2 = novo_inicio + len("Passag-")    
                    data_ref_arquivo = datetime.date(int(arquivo[novo_inicio_2 : final][0:4]),
                                                     int(arquivo[novo_inicio_2 : final][4:6]),
                                                     int(arquivo[novo_inicio_2 : final][6:8]))  
                except:
                    try:
                        novo_inicio = arquivo.find("Pass_Transp_") + len("Pass_Transp_")
                        data_ref_arquivo = datetime.date(int(ano),
                                                         int(dict_chave_meses[arquivo[novo_inicio : final][0:3].upper()]),
                                                         int(arquivo[novo_inicio : final][3:6]))  
                    except:
                        #Nesses casos, podem haver erro na nomemclatura do arquivo (Ex. 201300901)
                        novo_inicio = arquivo.find("Passag-") + len("Passag-")
                        data_ref_arquivo = datetime.date(int(arquivo[novo_inicio : final][0:4]),
                                                         int(arquivo[novo_inicio : final][5:7]),
                                                         int(arquivo[novo_inicio : final][7:9]))

            
            #Formato: "Passag-AAAAMMDD"
            elif (len(arquivo[inicio : final]) == 59) or (len(arquivo[inicio : final]) == 61) or  (len(arquivo[inicio : final]) == 64):
                novo_inicio = (arquivo.find("Passag-") + len("Passag-"))
                data_ref_arquivo = datetime.date(int(arquivo[novo_inicio : final][0:4]),
                                                     int(arquivo[novo_inicio : final][4:6]),
                                                     int(arquivo[novo_inicio : final][6:8]))
                
            
            #Formato: "Passag-Transportados-AAAAMMDD"
            elif (len(arquivo[inicio : final]) == 72) or (len(arquivo[inicio : final]) == 73):
                novo_inicio = (arquivo.find("Passag-Transportados") + len("Passag-Transportados"))
                data_ref_arquivo = datetime.date(int(arquivo[novo_inicio : final][1:5]),
                                                     int(arquivo[novo_inicio : final][5:7]),
                                                     int(arquivo[novo_inicio : final][7:9]))  
                                    
            
            #Formato: Novo diretório para download "arquivo[0:10] == '/documents'"
            elif arquivo[0:10] == '/documents':
                novo_arquivo = arquivo[len('/documents/d/mobilidade/') : len(arquivo)-4]
                data_ref_arquivo = datetime.date(int(novo_arquivo[5:9]),
                                                 dict_chave_meses[novo_arquivo[2:5].upper()],
                                                 int(novo_arquivo[0:2]))
            
            
            else:
                data_ref_arquivo = None


            if data_ref_arquivo != None:
                if arquivo[0:10] == '/documents':
                    arquivo = 'https://capital.sp.gov.br' + arquivo
                    
                dict_arquivos_disponiveis[data_ref_arquivo] = [arquivo, "Não Baixado"]
            
            
        return dict_arquivos_disponiveis

    #Limpo o diretorio temporário de arquivos do tipo .ZIP e .TXT
    def limpar_Diretorio(self):
        arquivos_temp = glob.glob(diretorio + '/temp_downloads/*')
        for temp in arquivos_temp:
            os.remove(temp)

    
    def download_Arquivos(self, dict_log_datas, dict_arquivos_disponiveis, ano):
        """
        Método responsável pelo download dos arquivos.
        Data uma lista de datas e de um dincionário de arquivos disponíveis no site,
        ele irá fazer o download e escrever um log de download
        """                
        
        for data, log in dict_log_datas.items():
            if data.year == ano:
                try:
                    #Tento fazer o download no diretório temporário
                    wget.download(dict_arquivos_disponiveis[data][0],
                                                                    out = diretorio + "/temp_downloads/Bilhetagem_" 
                                                                                                            + str(data.year) + "_" 
                                                                                                            + str(data.month) + "_"
                                                                                                            + str(data.day) 
                                                                                                            + ".xls")
                    
                    time.sleep(2)
                    
                    #Atualizo o log
                    dict_log_datas[data] = "Baixado"
                    dict_arquivos_disponiveis[data][1] = "Baixado"
                except BaseException as e:
                    try:
                        dict_log_datas[data] = str(e)
                        dict_arquivos_disponiveis[data][1] = str(e)
                    except:
                        dict_log_datas[data] = 'Sem referencia'
                        #dict_arquivos_disponiveis[data][1] = 'Sem referencia'
            else:
                pass
        
        return dict_log_datas
    
    def salvar_log(self, dict_log_datas):
        dic_log = {}
        i = 0
        
        for data, log in dict_log_datas.items():
            dic_log[i] = [str(data), log]
            i = i +1        
        
        try:
            with open(diretorio + "\log_download_bilhetagem.json", "w") as log_file:
                json.dump(dic_log , log_file)
            print("Log de download salvo no diretório: ", diretorio + "/log_download_bilhetagem.json" )
        except BaseException as e:
            print("ERRO! Log de download não pode ser salvo! " + str(e))

            
            
    def atualizar_log(self, dict_log_datas, dict_log_datas_novo):
        dic_log = {}
        i = 0
        
        #Adicono os ados do logo novo    
        for data, log in dict_log_datas_novo.items():
            dic_log[i] = [str(data), log]
            i = i +1     

        #Adiciono os dados do log antigo
        for chave, log in dict_log_datas.items():
            dic_log[i] = [log[0], log[1]]
            i = i +1

        try:
            with open(diretorio + "\log_download_bilhetagem.json", "w") as log_file:
                json.dump(dic_log , log_file)
            print("Log de download salvo no diretório: ", diretorio + "/log_download_bilhetagem.json" )
        except BaseException as e:
            print("ERRO! Log de download não pode ser salvo! " + str(e))            
            
                        
    def ler_log(self):
        with open(diretorio + "\log_download_bilhetagem.json") as dict_log_datas:
            dict_log_datas = json.load(dict_log_datas)
        return dict_log_datas

In [10]:
class consolidador_Arquivos(object):    
    def listar_Arquivos(self):        
        arquivos_temp = glob.glob(diretorio + '/temp_downloads/*')        
        return arquivos_temp
    
    """
    Esse método carrega o DF já disponível com os dados
    Se não houver um arquivo disponível, ele criará um vazio    
    """
    def criar_Df_Bilhetagem(self):
        #Se o BD existir
        if os.path.isfile(diretorio + "/BD_Bilhetagem.csv") == True:
            df = pd.read_csv(diretorio + "/BD_Bilhetagem.csv", encoding='latin-1')
            
        else:
            df = pd.DataFrame(columns=self.ordem_Colunas())
            print("Um banco de dados não pode ser encotrando em " + diretorio)
            print("Um novo banco de dados está sendo criado para a extração corrente...")
        return df
    
    def carregar_Arquivo(self, arquivo):
        #dic_colunas_depara = self.carregar_DePara_Colunas()
        
        try:
            i = 0 
            plan = pd.read_excel(arquivo, skiprows = range(0, i))
            while plan.columns[0].upper()[0:9] == 'RELATÓRIO' or plan.columns[0] == 'Unnamed: 0':
                i = i + 1
                plan = pd.read_excel(arquivo, skiprows = range(0, i))

            
            #Defino os novos nomes da coluna, baseado no De-Para
            colunas = (plan.columns.str.upper().str.strip()).to_series().map(self.carregar_DePara_Colunas())
            plan = plan.set_axis(colunas, axis=1)
            
            #As colunas que não estão no De-Para são excluídas
            plan.columns = plan.columns.fillna('excluir')
            plan.drop('excluir', axis = 1, inplace = True)
            
            #Para garatnir consistência, i) excluo o coluna de DATA e...           
            plan = plan.drop(['DATA', 'Data'], axis=1, errors='ignore')
            
            #ii) Crio uma nova coluna de Data (como nome do arquivo)
            data_ref = arquivo[arquivo.find('Bilhetagem_')+len('Bilhetagem_'):arquivo.find('.xls')]
            plan['DATA'] = datetime.datetime.strptime(data_ref, '%Y_%m_%d')
            
            #Crio uma nova coluna como Nome do Arquivo            
            plan['NOME_ARQUIVO'] = arquivo
                                    
            plan = plan.reindex(self.ordem_Colunas(), axis=1)
                        
            return True, plan
        
        except BaseException as e:   
            print("ERRO! Não foi possível carregar arquivo " + str(arquivo))
            print(str(e))
            return False, None
        
        
    def carregar_DePara_Colunas(self):
        try:
            with open(diretorio + '/De_Para_Colunas.json') as de_para:
                dic_colunas_depara  = json.load(de_para)
                return dic_colunas_depara
        except BaseException as e:
            print("ERRO! Não foi possível carregar o De-Para de Colunas! " + str(e))     
            
            
    def ordem_Colunas(self):        
        ordem_colunas = ['NOME_ARQUIVO', 'DATA', 'AREA', 'EMPRESA', 'LINHA', 'TIPO', 'PASSAGEIROS_GRATUIDADE', 
            'PASSAGEIROS_GRATUIDADE_ESTUDANTE', 'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM', 
            'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES', 
            'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP', 
            'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL', 'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES', 
            'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS', 
            'TOTAL']
        
        return ordem_colunas
        
                
    '''
    PROCESSO PARA RECARREGAMENTO DOS DADOS
    '''    
    def definir_Colunas(self, Lista_Colunas):
        Cols_Remover = ['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8',
                        'Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16',
                        'Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24',
                        'Unnamed: 25','Unnamed: 26','Unnamed: 27', 'DATA','Data']
        
        Cols_Manter = []
        for col in list(set(Lista_Colunas)):
            if col not in Cols_Remover:
                Cols_Manter.append(col)
        return Cols_Manter

## Instanciação das Classes

In [11]:
Crawler = None
Gerenciador = None
Consolidador = None

Crawler = Classe_Geral()
Gerenciador = gerenciador_Download()
Consolidador = consolidador_Arquivos()

## Execução das Rotinas

## Atualização do BD

#### i. Download dos arquivos não disponíveis

In [ ]:
"""
Para o caso de fazer download 
apenas das datas não presentes no BD
"""
if Download_BD_Atualizacao == 1:
    dict_log_datas = Gerenciador.ler_log()
        
    lista_anos, result_set = Crawler.carregar_Anos_Consulta()

    #Crio arquivo responsável pelo log
    dict_log_datas_novo = Crawler.criar_Relacao_Datas_Atualizacao(dict_log_datas)
    
    #Menor ano do novo dicionário de log
    ano = min(dict_log_datas_novo.keys()).year
    print("Carregando páginas do ano ", str(ano), "\n")    
    
    
    """CRAWLER"""    
    #i) Carrego a url pertinente
    url = Crawler.carregar_Href_Ano(str(ano), result_set)
    
    #ii)Capturo as urls dos arquivos deste ano
    lista_arquivos = Crawler.carregar_Href_datas(url)
    
    """GERENCIADOR"""    
    #iv)Mapeio os arquivos
    print("Mapeando arquivos para o ano ", str(ano), "\n")
    dict_arquivos_disponiveis = Gerenciador.mapeamento_Arquivos(lista_arquivos, ano)

    #v)Faço o download dos arquivos no diretório temporárop
    print("Iniciando downloads...", "\n")
    Gerenciador.limpar_Diretorio()
    dict_log_datas_novo = Gerenciador.download_Arquivos(dict_log_datas_novo, dict_arquivos_disponiveis, ano)
    
    Gerenciador.atualizar_log(dict_log_datas, dict_log_datas_novo)

#### ii. Consolidação dos Dados Historicos em um Novo Df_Bilhetagem <font color='red'>(Df_Bilhetagem será sobrescrito) </font>

In [ ]:
"""Criando DF vazio"""
Df_Bilhetagem = Consolidador.criar_Df_Bilhetagem()

"""Listando arquivos temporários"""
arquivos_temp = Consolidador.listar_Arquivos()

arquivos_erro = []
"""Recuperando arquivos na lista e fazendo o append à vazia"""
for arquivo in arquivos_temp:    
    carregamento, df_temp = Consolidador.carregar_Arquivo(arquivo)    
    if carregamento == True:
        Df_Bilhetagem = pd.concat([Df_Bilhetagem, df_temp], axis=0, ignore_index=False)
    else:
        arquivos_erro.append(arquivo)
        print("Erro ao carregar ", str(arquivo))
        
if len(arquivos_erro) >0:
    print("Existem  ", str(len(arquivos_erro)), " arquivos para serem corrigidos antes de prosseguir")

#### Para caso de Erros (Corrija a coluna A dos arquivos temporários que deram erro (Tabular com Texto)

In [ ]:
if len(arquivos_erro) > 0:
    """Recuperando arquivos na lista e fazendo o append à vazia"""
    for arquivo in arquivos_erro:    
        carregamento, df_temp = Consolidador.carregar_Arquivo(arquivo)    
        if carregamento == True:
            Df_Bilhetagem = pd.concat([Df_Bilhetagem, df_temp], axis=0, ignore_index=False)
        else:
            print("Erro ao carregar ", str(arquivo))

#### Totalização e Salvamento

In [ ]:
#Totalização dos Dados        
cols_total = ['PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE', 'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM', 'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL', 'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS']
Df_Bilhetagem['TOTAL'] = Df_Bilhetagem[cols_total].sum(axis=1)

#Salvar o Arquivo
Df_Bilhetagem.to_csv(diretorio + '/BD_Bilhetagem.csv', encoding='latin-1', index=False)        

## Dados Históricos

#### ii. Download dos Dados Historicos  <font color='red'>(Todos os arquivos temporários serão excluídos)</font>

In [ ]:
"""
Para o caso de gerar uma 
nova extração histórica
"""
if Download_BD_Historico == 1:
    lista_anos, result_set = Crawler.carregar_Anos_Consulta()

    #Os anos de 2014 e 2013 estão em páginas separadas, então faço o append separadamente
    lista_anos.append(2014)
    lista_anos.append(2013)

    #Crio arquivo responsável pelo log
    dict_log_datas = Crawler.criar_Relacao_Datas(lista_anos)
    
    Gerenciador.limpar_Diretorio()

    #Para cada ano...
    for ano in lista_anos:
##    if ano == 2024 and ano != 2023:
        print("Carregando páginas do ano ", str(ano), "\n")
        """CRAWLER"""    
        #i) Carrego a url pertinente
        url = Crawler.carregar_Href_Ano(str(ano), result_set)
        #print(url)

        #ii)Capturo as urls dos arquivos deste ano
        lista_arquivos = Crawler.carregar_Href_datas(url)

        """GERENCIADOR"""
        #iv)Mapeio os arquivos
        print("Mapeando arquivos para o ano ", str(ano), "\n")
        dict_arquivos_disponiveis = Gerenciador.mapeamento_Arquivos(lista_arquivos, ano)

        #v)Faço o download dos arquivos no diretório temporárop
        print("Iniciando downloads...", "\n")
        dict_log_datas = Gerenciador.download_Arquivos(dict_log_datas, dict_arquivos_disponiveis, ano)

    Gerenciador.salvar_log(dict_log_datas)

#### ii. Consolidação dos Dados Historicos em um Novo Df_Bilhetagem <font color='red'>(Df_Bilhetagem será sobrescrito) </font>

In [13]:
"""Criando DF vazio"""
Df_Bilhetagem = Consolidador.criar_Df_Bilhetagem()

"""Listando arquivos temporários"""
arquivos_temp = Consolidador.listar_Arquivos()


arquivos_erro = []
"""Recuperando arquivos na lista e fazendo o append à vazia"""
for arquivo in arquivos_temp:    
    carregamento, df_temp = Consolidador.carregar_Arquivo(arquivo)    
    if carregamento == True:
        Df_Bilhetagem = pd.concat([Df_Bilhetagem, df_temp], axis=0, ignore_index=False)
    else:
        print("Erro ao carregar ", str(arquivo))
        
if len(arquivos_erro) >0:
    print("Existem  ", str(len(arquivos_erro)), " arquivos para serem corrigidos antes de prosseguir")

Um banco de dados não pode ser encotrando em C:\Users\f112596\Desktop\Projetos\DO\Lab Transp
Um novo banco de dados está sendo criado para a extração corrente...
ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2013_10_13.xls
"['excluir'] not found in axis"
Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2013_10_13.xls
ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2013_7_13.xls
"['excluir'] not found in axis"
Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2013_7_13.xls
ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2015_3_15.xls
"['excluir'] not found in axis"
Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2015_3_15.xls
ERRO! Não foi possível carregar arquiv

ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_12_30.xls

Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_12_30.xls
ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_1_19.xls

Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_1_19.xls
ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_1_31.xls

Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_1_31.xls
ERRO! Não foi possível carregar arquivo C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_2_20.xls

Erro ao carregar  C:\Users\f112596\Desktop\Projetos\DO\Lab Transp/temp_downloads\Bilhetagem_2019_2_20.xls
ERRO! Não foi possível carregar arquivo C:\Users\f112596\D

#### Para caso de Erros (Corrija a coluna A dos arquivos temporários que deram erro)

In [14]:
if len(arquivos_erro) > 0:
    """Recuperando arquivos na lista e fazendo o append à vazia"""
    for arquivo in arquivos_erro:    
        carregamento, df_temp = Consolidador.carregar_Arquivo(arquivo)    
        if carregamento == True:
            Df_Bilhetagem = pd.concat([Df_Bilhetagem, df_temp], axis=0, ignore_index=False)
        else:
            print("Erro ao carregar ", str(arquivo))   

In [15]:
arquivos_erro

[]

#### Totalização e Salvamento

In [16]:
#Totalização dos Dados        
cols_total = ['PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE', 'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM', 'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES', 'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP', 'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL', 'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM', 'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS']
Df_Bilhetagem['TOTAL'] = Df_Bilhetagem[cols_total].sum(axis=1)

#Salvar o Arquivo
Df_Bilhetagem.to_csv(diretorio + '/BD_Bilhetagem.csv', encoding='latin-1', index=False)        

# Visualização dos Dados

In [ ]:
try:
    if 'Df_Bilhetagem' not in locals() or Df_Bilhetagem==None:
        consolidador = consolidador_Arquivos()
        Df_Bilhetagem = consolidador.criar_Df_Bilhetagem()
except:
    pass

In [ ]:
#Tratamento do Campo DATA
Df_Bilhetagem['DATA'] = Df_Bilhetagem['DATA'].astype(str)
Df_Bilhetagem['DATA'] = Df_Bilhetagem['DATA'].str.replace(" 00:00:00","")
Df_Bilhetagem['DATA'] = pd.to_datetime(Df_Bilhetagem['DATA'], format="%Y-%m-%d")
Df_Bilhetagem['ANO'] = pd.DatetimeIndex(Df_Bilhetagem['DATA']).year
Df_Bilhetagem['MES'] = pd.DatetimeIndex(Df_Bilhetagem['DATA']).month
Df_Bilhetagem['DIA'] = pd.DatetimeIndex(Df_Bilhetagem['DATA']).day

In [ ]:
#Agrupamento por Ano/Mês/Linha
Df_AnoMesLinha = Df_Bilhetagem[['ANO', 'MES', 'LINHA', 'PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE',
                                'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM',
                                'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL',
                                'PASSAGEIROS_PAGANTES_BU_ESTUDANTES',
                                'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL',
                                'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP',
                                'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL',
                                'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES',
                                'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM',
                                'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS', 'TOTAL']].groupby(as_index=False, by=['ANO', 'MES', 'LINHA']).sum()
Df_AnoMesLinha.head(5)

In [ ]:
#Agrupamento por Ano/Linha
Df_AnoLinha = Df_Bilhetagem[['ANO', 'LINHA', 'PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE',
                             'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM',
                             'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL',
                             'PASSAGEIROS_PAGANTES_BU_ESTUDANTES',
                             'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL',
                             'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP',
                             'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL',
                             'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES',
                             'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM',
                             'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS',
                             'TOTAL']].groupby(as_index=False, by=['ANO', 'LINHA']).sum()
Df_AnoLinha.head(5)

In [ ]:
#Agrupamento por Mês/Linha
Df_MesLinha = Df_Bilhetagem[['MES', 'LINHA', 'PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE',
                             'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM',
                             'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL',
                             'PASSAGEIROS_PAGANTES_BU_ESTUDANTES',
                             'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL',
                             'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP',
                             'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL',
                             'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES',
                             'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM',
                             'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS',
                             'TOTAL']].groupby(as_index=False, by=['MES', 'LINHA']).sum()
Df_MesLinha.head(5)

In [ ]:
#Agrupamento por Ano/Mês
Df_AnoMes = Df_Bilhetagem[['ANO', 'MES', 'PASSAGEIROS_GRATUIDADE', 'PASSAGEIROS_GRATUIDADE_ESTUDANTE',
                           'PASSAGEIROS_PAGANTES', 'PASSAGEIROS_PAGANTES_BU_COMUM',
                           'PASSAGEIROS_PAGANTES_BU_COMUM_MENSAL',
                           'PASSAGEIROS_PAGANTES_BU_ESTUDANTES',
                           'PASSAGEIROS_PAGANTES_BU_ESTUDANTES_MENSAL',
                           'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP',
                           'PASSAGEIROS_PAGANTES_BU_VALE_TRANSP_MENSAL',
                           'PASSAGEIROS_PAGANTES_DINHEIRO', 'PASSAGEIROS_PAGANTES_ESTUDANTES',
                           'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_METRO_CPTM',
                           'PASSAGEIROS_PAGANTES_INTEIRO_INTEGRACAO_ONIBUS',
                           'TOTAL']].groupby(as_index=False, by=['ANO', 'MES']).sum()
Df_AnoMes.head(5)

In [ ]:
Df_Bilhetagem = None